In [1]:
from __future__ import print_function

from pyalgotrade import strategy
from pyalgotrade.barfeed import quandlfeed
from pyalgotrade.technical import ma

In [2]:
class MyStrategy(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument, smaPeriod):
        super(MyStrategy, self).__init__(feed, 1000)
        self.__position = None
        self.__instrument = instrument
        # We'll use adjusted close values instead of regular close values.
        self.setUseAdjustedValues(True)
        self.__sma = ma.SMA(feed[instrument].getPriceDataSeries(), smaPeriod)

    def onEnterOk(self, position):
        execInfo = position.getEntryOrder().getExecutionInfo()
        self.info("BUY at $%.2f" % (execInfo.getPrice()))

    def onEnterCanceled(self, position):
        self.__position = None

    def onExitOk(self, position):
        execInfo = position.getExitOrder().getExecutionInfo()
        self.info("SELL at $%.2f" % (execInfo.getPrice()))
        self.__position = None

    def onExitCanceled(self, position):
        # If the exit was canceled, re-submit it.
        self.__position.exitMarket()

    def onBars(self, bars):
        # Wait for enough bars to be available to calculate a SMA.
        if self.__sma[-1] is None:
            return

        bar = bars[self.__instrument]
        # If a position was not opened, check if we should enter a long position.
        if self.__position is None:
            if bar.getPrice() > self.__sma[-1]:
                # Enter a buy market order for 10 shares. The order is good till canceled.
                self.__position = self.enterLong(self.__instrument, 10, True)
        # Check if we have to exit the position.
        elif bar.getPrice() < self.__sma[-1] and not self.__position.exitActive():
            self.__position.exitMarket()

In [4]:
def run_strategy(smaPeriod):
    # Load the bar feed from the CSV file
    feed = quandlfeed.Feed()
    feed.addBarsFromCSV("orcl", "WIKI-ORCL.csv")

    # Evaluate the strategy with the feed.
    myStrategy = MyStrategy(feed, "orcl", smaPeriod)
    myStrategy.run()
    print("Final portfolio value: $%.2f" % myStrategy.getBroker().getEquity())

run_strategy(20)

1986-04-14 00:00:00 strategy [INFO] BUY at $0.06
1986-05-05 00:00:00 strategy [INFO] SELL at $0.07
1986-05-06 00:00:00 strategy [INFO] BUY at $0.07
1986-05-20 00:00:00 strategy [INFO] SELL at $0.07
1986-06-12 00:00:00 strategy [INFO] BUY at $0.07
1986-06-13 00:00:00 strategy [INFO] SELL at $0.07
1986-06-16 00:00:00 strategy [INFO] BUY at $0.07
1986-06-18 00:00:00 strategy [INFO] SELL at $0.07
1986-06-20 00:00:00 strategy [INFO] BUY at $0.07
1986-06-23 00:00:00 strategy [INFO] SELL at $0.07
1986-06-24 00:00:00 strategy [INFO] BUY at $0.07
1986-06-30 00:00:00 strategy [INFO] SELL at $0.07
1986-07-02 00:00:00 strategy [INFO] BUY at $0.07
1986-07-03 00:00:00 strategy [INFO] SELL at $0.07
1986-08-19 00:00:00 strategy [INFO] BUY at $0.04
1986-08-21 00:00:00 strategy [INFO] SELL at $0.04
1986-08-26 00:00:00 strategy [INFO] BUY at $0.04
1986-08-27 00:00:00 strategy [INFO] SELL at $0.04
1986-08-29 00:00:00 strategy [INFO] BUY at $0.04
1986-09-09 00:00:00 strategy [INFO] SELL at $0.04
1986-09-18

1991-07-09 00:00:00 strategy [INFO] BUY at $0.20
1991-07-11 00:00:00 strategy [INFO] SELL at $0.19
1991-07-12 00:00:00 strategy [INFO] BUY at $0.20
1991-07-16 00:00:00 strategy [INFO] SELL at $0.20
1991-07-17 00:00:00 strategy [INFO] BUY at $0.20
1991-08-20 00:00:00 strategy [INFO] SELL at $0.21
1991-08-22 00:00:00 strategy [INFO] BUY at $0.23
1991-11-15 00:00:00 strategy [INFO] SELL at $0.33
1991-12-17 00:00:00 strategy [INFO] BUY at $0.30
1991-12-18 00:00:00 strategy [INFO] SELL at $0.29
1991-12-19 00:00:00 strategy [INFO] BUY at $0.29
1991-12-20 00:00:00 strategy [INFO] SELL at $0.30
1991-12-24 00:00:00 strategy [INFO] BUY at $0.29
1992-03-02 00:00:00 strategy [INFO] SELL at $0.44
1992-04-23 00:00:00 strategy [INFO] BUY at $0.31
1992-04-24 00:00:00 strategy [INFO] SELL at $0.30
1992-05-01 00:00:00 strategy [INFO] BUY at $0.31
1992-05-26 00:00:00 strategy [INFO] SELL at $0.33
1992-05-29 00:00:00 strategy [INFO] BUY at $0.34
1992-06-01 00:00:00 strategy [INFO] SELL at $0.33
1992-06-03

1997-01-15 00:00:00 strategy [INFO] BUY at $4.38
1997-01-16 00:00:00 strategy [INFO] SELL at $4.21
1997-01-21 00:00:00 strategy [INFO] BUY at $4.31
1997-01-22 00:00:00 strategy [INFO] SELL at $4.19
1997-02-11 00:00:00 strategy [INFO] BUY at $4.03
1997-02-12 00:00:00 strategy [INFO] SELL at $4.02
1997-02-13 00:00:00 strategy [INFO] BUY at $4.21
1997-02-28 00:00:00 strategy [INFO] SELL at $3.99
1997-03-17 00:00:00 strategy [INFO] BUY at $4.07
1997-04-01 00:00:00 strategy [INFO] SELL at $3.67
1997-04-04 00:00:00 strategy [INFO] BUY at $3.82
1997-04-10 00:00:00 strategy [INFO] SELL at $3.85
1997-04-25 00:00:00 strategy [INFO] BUY at $3.93
1997-06-23 00:00:00 strategy [INFO] SELL at $4.90
1997-06-25 00:00:00 strategy [INFO] BUY at $5.07
1997-07-02 00:00:00 strategy [INFO] SELL at $4.90
1997-07-09 00:00:00 strategy [INFO] BUY at $5.15
1997-08-18 00:00:00 strategy [INFO] SELL at $5.61
1997-08-19 00:00:00 strategy [INFO] BUY at $5.95
1997-08-28 00:00:00 strategy [INFO] SELL at $5.78
1997-09-03

2001-12-14 00:00:00 strategy [INFO] SELL at $13.28
2001-12-18 00:00:00 strategy [INFO] BUY at $13.83
2001-12-21 00:00:00 strategy [INFO] SELL at $13.50
2002-01-04 00:00:00 strategy [INFO] BUY at $14.36
2002-02-04 00:00:00 strategy [INFO] SELL at $15.04
2002-02-26 00:00:00 strategy [INFO] BUY at $14.76
2002-03-04 00:00:00 strategy [INFO] SELL at $12.61
2002-05-17 00:00:00 strategy [INFO] BUY at $8.83
2002-05-21 00:00:00 strategy [INFO] SELL at $8.29
2002-05-24 00:00:00 strategy [INFO] BUY at $8.31
2002-05-28 00:00:00 strategy [INFO] SELL at $8.18
2002-05-29 00:00:00 strategy [INFO] BUY at $7.99
2002-05-30 00:00:00 strategy [INFO] SELL at $7.55
2002-06-17 00:00:00 strategy [INFO] BUY at $7.86
2002-06-24 00:00:00 strategy [INFO] SELL at $7.26
2002-06-25 00:00:00 strategy [INFO] BUY at $7.87
2002-07-23 00:00:00 strategy [INFO] SELL at $8.46
2002-07-30 00:00:00 strategy [INFO] BUY at $8.61
2002-08-05 00:00:00 strategy [INFO] SELL at $8.40
2002-08-09 00:00:00 strategy [INFO] BUY at $8.88
200

2007-01-05 00:00:00 strategy [INFO] BUY at $15.92
2007-01-12 00:00:00 strategy [INFO] SELL at $15.68
2007-01-16 00:00:00 strategy [INFO] BUY at $15.72
2007-01-17 00:00:00 strategy [INFO] SELL at $15.63
2007-01-18 00:00:00 strategy [INFO] BUY at $15.80
2007-01-19 00:00:00 strategy [INFO] SELL at $15.42
2007-02-05 00:00:00 strategy [INFO] BUY at $15.68
2007-02-06 00:00:00 strategy [INFO] SELL at $15.48
2007-02-21 00:00:00 strategy [INFO] BUY at $15.34
2007-02-26 00:00:00 strategy [INFO] SELL at $15.28
2007-03-07 00:00:00 strategy [INFO] BUY at $15.16
2007-03-08 00:00:00 strategy [INFO] SELL at $15.09
2007-03-13 00:00:00 strategy [INFO] BUY at $15.30
2007-03-14 00:00:00 strategy [INFO] SELL at $15.13
2007-03-15 00:00:00 strategy [INFO] BUY at $15.23
2007-03-16 00:00:00 strategy [INFO] SELL at $15.10
2007-03-20 00:00:00 strategy [INFO] BUY at $15.59
2007-05-02 00:00:00 strategy [INFO] SELL at $16.89
2007-05-03 00:00:00 strategy [INFO] BUY at $17.09
2007-05-10 00:00:00 strategy [INFO] SELL 

2011-12-05 00:00:00 strategy [INFO] SELL at $29.40
2011-12-06 00:00:00 strategy [INFO] BUY at $29.61
2011-12-09 00:00:00 strategy [INFO] SELL at $28.50
2011-12-12 00:00:00 strategy [INFO] BUY at $29.02
2011-12-14 00:00:00 strategy [INFO] SELL at $28.35
2012-01-13 00:00:00 strategy [INFO] BUY at $25.05
2012-02-13 00:00:00 strategy [INFO] SELL at $26.48
2012-02-17 00:00:00 strategy [INFO] BUY at $26.89
2012-02-23 00:00:00 strategy [INFO] SELL at $26.48
2012-02-24 00:00:00 strategy [INFO] BUY at $26.92
2012-03-22 00:00:00 strategy [INFO] SELL at $27.17
2012-04-09 00:00:00 strategy [INFO] BUY at $27.01
2012-04-10 00:00:00 strategy [INFO] SELL at $26.81
2012-04-18 00:00:00 strategy [INFO] BUY at $27.04
2012-04-23 00:00:00 strategy [INFO] SELL at $26.55
2012-04-27 00:00:00 strategy [INFO] BUY at $26.97
2012-05-07 00:00:00 strategy [INFO] SELL at $26.31
2012-06-06 00:00:00 strategy [INFO] BUY at $25.05
2012-07-25 00:00:00 strategy [INFO] SELL at $27.25
2012-07-27 00:00:00 strategy [INFO] BUY 

2016-08-12 00:00:00 strategy [INFO] BUY at $40.33
2016-08-25 00:00:00 strategy [INFO] SELL at $40.32
2016-08-29 00:00:00 strategy [INFO] BUY at $40.67
2016-09-02 00:00:00 strategy [INFO] SELL at $40.45
2016-09-06 00:00:00 strategy [INFO] BUY at $40.56
2016-09-09 00:00:00 strategy [INFO] SELL at $39.54
2016-11-01 00:00:00 strategy [INFO] BUY at $37.97
2016-11-03 00:00:00 strategy [INFO] SELL at $37.39
2016-11-04 00:00:00 strategy [INFO] BUY at $37.71
2016-12-02 00:00:00 strategy [INFO] SELL at $37.85
2016-12-08 00:00:00 strategy [INFO] BUY at $39.68
2016-12-19 00:00:00 strategy [INFO] SELL at $38.41
2017-01-12 00:00:00 strategy [INFO] BUY at $38.56
2017-04-07 00:00:00 strategy [INFO] SELL at $43.56
2017-04-19 00:00:00 strategy [INFO] BUY at $44.10
2017-04-20 00:00:00 strategy [INFO] SELL at $43.92
2017-04-21 00:00:00 strategy [INFO] BUY at $44.25
2017-05-18 00:00:00 strategy [INFO] SELL at $43.72
2017-05-26 00:00:00 strategy [INFO] BUY at $44.86
2017-06-12 00:00:00 strategy [INFO] SELL 

Final portfolio value: $1030.32
